In [7]:
# IMPORTS
from enron.src.common.const import *
from enron.src.classification_algorithms.assign_management_levels import *
from enron.src.visualization.random_baseline_enron import *

from manufacturing_company.src.common.const import *
from manufacturing_company.src.network.social_network import *
from manufacturing_company.src.classification_algorithms.collective_classification import *
from manufacturing_company.src.logs.collective_classification_logger import CollectiveClassificationLogger
from manufacturing_company.src.visualization.plot_collective_classification import PlotCollectiveClassification

import warnings
warnings.filterwarnings(action='once')


In [8]:
# CONST
pcts = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
thresholds = [2, 3, 4, 5, 6, 7, 8, 9, 10]
jaccard_mins = [0.7, 0.8, 0.9, 0.99]
levels = 2
minority_labels = [] #[1, 2]

positions = pd.read_csv(ENRON_FILE_FLATTEN_HIERARCHY, sep=';', index_col=ID)
positions = positions[[FLATTEN_POSITION]]
positions.columns = [POSITION]

In [1]:
# COLLECTIVE CLASSIFICATION

logger = CollectiveClassificationLogger('enron', levels)

for month in range(MONTHS):
    features = pd.read_csv(ENRON_FILE_FEATURES.format(month), sep=';', index_col=ID)
    features = assign_management_levels_cc(levels, features, positions)

    emails = pd.read_csv(ENRON_FILE_MINIMUM_ACTIVITY.format(month), sep=';')
    G = create_network(emails, weight=False)
    G_undirected = G.to_undirected()

    for pct in pcts:
        for threshold in thresholds:
            for jaccard_min in jaccard_mins:
                collective_classification(logger, month, G_undirected, features.copy(), pct, levels, threshold, minority_labels, jaccard_min)

plot = PlotCollectiveClassification(logger.directory_path, levels, random_baseline_enron)
plot.plot()